今天我们会学习从豆瓣上爬取数据，帮助静静解决问题。你将会完成以下内容：

1. 向网页发送请求，获取网页源代码;
2. 解析源代码，提取想要的数据；
3. 使用 jieba 模块将语句切分成词；
4. 运用字典的知识进行词频统计；
5. 将词语生成词云图。

今天我们完成前两个步骤。后面将会学习剩下的步骤。

# 分析网页

想要获取网页中的短评，首先要获取网页 HTML 代码，再把短评从中提取出来。

在之前的课程中学习过，我们要向网页的服务器发送请求，服务器返回的响应就是网页 HTML 代码。

想要获取网页中的 HTML 代码，首先，我们需要使用 import 导入 requests 模块。

将豆瓣电影评论的 URL 地址👇，赋值给变量url。
https://movie.douban.com/subject/2129039/comments?sort=new_score&status=P

接着，将变量 url 传入requests.get()，向对应的服务器发送 GET 请求，返回的 Response 赋值给变量 response

In [1]:
# TODO 使用import导入requests模块
import requests

# TODO 将豆瓣电影评论URL地址，赋值给变量url
url = "https://movie.douban.com/subject/2129039/comments?sort=new_score&status=P"

# TODO 使用requests发起GET请求，赋值给response
response = requests.get(url)

# TODO 使用print输出response
print(response)

<Response [418]>


状态码418
欸？返回一个状态码 418(°ー°〃)

418 其实是一个愚人节玩笑，含义是：当客户端给一个茶壶发送泡咖啡的请求时，茶壶就返回一个 418 错误状态码，表示“我是一个茶壶”。

在这里，状态码 418 也意味着我们的爬虫被豆瓣“发现”了，这该怎么办呢？

# 请求网页

## User-Agent

在进行 HTTP 请求的时候，除了请求指定的 url 信息之外，还会告诉服务端“我是谁”，“我支持哪些特性”。

这些信息就是 HTTP 请求头 ，其中声明“我是谁”的就是 HTTP 请求头部的 User-Agent 。

由于我们用的是 requests 发起的请求，而不是浏览器，所以我们需要伪装成 Chrome 浏览器。

在网页中请求头和响应头信息保存在变量 headers 中，查看请求头或响应头的方法为：

1. 使用 响应消息.request.headers 就能查看请求头中的信息。
2. 使用 响应消息.headers 就能查看响应头中的信息。

接下来，请求链接
https://nocturne-spider.baicizhan.com/2020/08/07/1/
获取响应消息，输出请求头中的内容。

In [2]:
# 使用import导入requests模块
import requests

# 从bs4中导入BeautifulSoup
from bs4 import BeautifulSoup

# 将URL地址赋值给变量url
url = "https://nocturne-spider.baicizhan.com/2020/08/07/1/"

# 将变量url传入requests.get()，赋值给r
r = requests.get(url)

# TODO 使用print()输出响应消息中的request.headers
print(r.request.headers)

{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


请求头中的 User-Agent 用来表示产生请求时浏览器的类型。

查看使用 响应消息.request.headers 输出的请求头中, User-Agent 的值为 python-requests ，说明此次请求是Python发出的并且能被服务器识别到。

那么怎么伪装成浏览器发出请求呢？接下来，学习浏览器中 User-Agent 的样式。

1. 设置请求头     

我们可以对请求头信息进行设置，请求头中有很多内容，常用的就是User-Agent。

请求头信息需要以字典key-value的形式赋值给字典 headers。    

2. headers参数      

在爬虫程序中添加请求头信息，也就是在请求链接时，带着参数headers。

在 requests.get() 中添加参数 headers，其参数值为字典 headers 。

现在我们模拟一个请求头信息， {"User-Agent":"JingJing"} 赋值给headers。

使用 get()函数请求链接时，带上headers参数，并使用 响应消息.request.headers 就能查看请求头中的信息。

查看输出结果，我们就已经修改了，那么浏览器 "User-Agent" 的值为多少呢？

In [3]:
# 使用import导入requests模块
import requests

# 从bs4中导入BeautifulSoup
from bs4 import BeautifulSoup

# 将URL地址赋值给变量url
url = "https://nocturne-spider.baicizhan.com/2020/08/07/1/"

# TODO 将User-Agent以字典键对形式赋值给headers
headers = {"User-Agent": "JingJing"}

# TODO 将字典headers传递给headers参数，添加进requests.get()中，赋值给r
r = requests.get(url, headers = headers)

# TODO 使用print()输出响应消息中的request.headers
print(r.request.headers)

{'User-Agent': 'JingJing', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


## 获取User-Agent

1. 打开网页，右键单击"检查"；    

2. 点击"Network"，并刷新网页；    

3. 找(第一个)文件，查看Request Headers的User-Agent

设置请求头      

我们用浏览器打开网页，就是浏览器对服务端发送的请求，刚刚查看的请求头就是浏览器的信息。

接下来，可以设置请求头信息，模拟成浏览器去访问网站。

复制浏览器的 User-Agent 的值，粘贴到代码中，以字典key-value的形式赋值给字典 headers。

在 requests.get() 中添加参数 headers，其参数值为字典 headers 。

在课程中，我们可以使用静静电脑浏览器中的 User-Agent          

User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36

在程序中添加请求头，将 User-Agent 以字典键对形式赋值给字典headers。

在 requests.get() 中添加 headers 参数，其参数值为字典 headers 。

接着将服务器响应内容以字符串形式输出，即可获得URL对应的网页内容。由于网页内容较多，我们这里可以用切片的方法，先输出前1000个字符。

In [4]:
# 使用import导入requests模块
import requests

# 将豆瓣电影评论URL地址，赋值给变量url
url = "https://movie.douban.com/subject/2129039/comments?sort=new_score&status=P"

# TODO 将User-Agent以字典键对形式赋值给headers
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}

# TODO 将字典headers传递给headers参数，添加进requests.get()中，赋值给response
response = requests.get(url, headers = headers)

# TODO 使用.text属性获取网页前1000个字符的内容，并赋值给html
html = response.text[:1000]

# TODO 使用print输出html
print(html)

<!DOCTYPE html>
<html lang="zh-CN" class="ua-mac ua-webkit">
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <meta name="renderer" content="webkit">
    <meta name="referrer" content="always">
    <meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />
    <title>

飞屋环游记 短评
</title>
    
    <meta name="baidu-site-verification" content="cZdR4xxR7RxmM4zE" />
    <meta http-equiv="Pragma" content="no-cache">
    <meta http-equiv="Expires" content="Sun, 6 Mar 2005 01:00:00 GMT">
    
    <meta name="keywords" content="飞屋环游记,Up,影讯,排片,放映时间,电影票价,在线购票"/>
    <meta name="description" content="飞屋环游记短评" />
    <meta name="mobile-agent" content="format=html5; url=https://m.douban.com/movie/subject/2129039/comments"/>
    <script type="text/javascript" src="https://img3.doubanio.com/f/shire/2c0c1c6b83f9a457b0f38c38a32fc43a42ec9bad/js/do.js" data-cfg-autoload="false"></script>

    <link rel="apple-touch-icon" href="https://i

# 解析网页

太好了，获取到网页 HTML 代码。接下来，就需要解析网页，从网页中提取想要的数据。

在上节课中我们学习了 BeautifulSoup 解析模块，创建一个 BeautifulSoup 对象；接着使用 find_all() 函数，提取出相关的节点。

我们要使用 BeautifulSoup 解析库，需要从 bs4 库中导入 BeautifulSoup

使用 BeautifulSoup() 函数，创建一个 BeautifulSoup 对象，传入变量 html 和解析器 lxml，将该对象赋值给变量 soup。    

```
# 使用import导入requests模块
import requests

# TODO 从bs4中导入BeautifulSoup
from bs4 import BeautifulSoup

# 将豆瓣电影评论URL地址，赋值给变量url
url = "https://movie.douban.com/subject/2129039/comments?sort=new_score&status=P"

# 将User-Agent以字典键对形式赋值给headers
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}

# 将 url 和 headers参数，添加进requests.get()中，将字典headers传递headers参数，给赋值给response
response = requests.get(url, headers=headers)

# 将服务器响应内容转换为字符串形式，赋值给html
html = response.text

# TODO 使用BeautifulSoup()传入变量html和解析器lxml，赋值给soup
soup = BeautifulSoup(html, "lxml")

# TODO 使用print输出soup
print(soup)
```

创建好 BeautifulSoup 对象后，接下来就要提取出短评所在的节点。

我们可以使用 BeautifulSoup 中的 find_all() 函数，它可以查询所有符合条件的元素，并组成一个列表。

在这里，find_all() 函数中传入的参数内容是什么呢？我们需要去网页中找到短评所在的节点。    

如何内容定位可以参考前面几章的笔记...

观察网页可以发现：

所有的评论都在` <span> `标签中，检索span标签在网页中出现了100多次，而显然，该页只有二十条评论。

也可以试试以` <p> `节点来定位，网页中p标签出现的次数也大于20。

那么我们该定位哪个节点，才能提取评论呢？    

发现了么，短评所在的节点都是类似的它们都有一样的标签和属性。

**在 HTML 中，span 标签的作用是用来组合文档中的行内元素。**

**class 属性的作用是用来给标签分类。class="short"也就代表着，这里标签的内容是短评。**

**在网页中属性需要使用 (点).属性值 来定位。**

例如，图中我们在检索框中输入 .short 进行搜索，就能看到该页中 class="short" 属性出现的次数为20次。

那么，就定位到了提取评论的节点 class="short" 。

以属性为查询条件就需要在 find_all() 中，传入 class="short"。

由于 class 在 Python 里是一个关键字 ，所以后面需要加一个下划线，即 `class_="short"`。 

```
# 使用import导入requests模块
import requests

# 从bs4中导入BeautifulSoup
from bs4 import BeautifulSoup

# 将豆瓣电影评论URL地址，赋值给变量url
url = "https://movie.douban.com/subject/2129039/comments?sort=new_score&status=P"

# 将User-Agent以字典键对形式赋值给headers
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}

# 将 url 和 headers参数，添加进requests.get()中，将字典headers传递headers参数，给赋值给response
response = requests.get(url, headers=headers)

# 将服务器响应内容转换为字符串形式，赋值给html
html = response.text

# 使用BeautifulSoup()传入变量html和解析器lxml，赋值给soup
soup = BeautifulSoup(html, "lxml")

# TODO 使用find_all()查询soup中class="short"的节点，赋值给content_all
content_all = soup.find_all(class_="short")

# TODO 使用print输出content_all
print(content_all)
```

由于 find_all() 返回的是一个列表，我们不能直接访问 .string 属性。

使用 for 循环遍历列表 content_all，访问 .string 获取每个节点中标签的内容，赋值给变量 contentString。

In [5]:
# 使用import导入requests模块
import requests

# 从bs4中导入BeautifulSoup
from bs4 import BeautifulSoup

# 将豆瓣电影评论URL地址，赋值给变量url
url = "https://movie.douban.com/subject/2129039/comments?sort=new_score&status=P"

# 将User-Agent以字典键对形式赋值给headers
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}

# 将 url 和 headers参数，添加进requests.get()中，将字典headers传递headers参数，给赋值给response
response = requests.get(url, headers=headers)

# 将服务器响应内容转换为字符串形式，赋值给html
html = response.text

# 使用BeautifulSoup()传入变量html和解析器lxml，赋值给soup
soup = BeautifulSoup(html, "lxml")

# 使用find_all()查询soup中class="short"的节点，赋值给content_all
content_all = soup.find_all(class_="short")

# TODO for循环遍历content_all
for content in content_all:

    # TODO 获取每个节点标签的内容，赋值给contentString
    contentString = content.string

    # TODO 使用print输出contentString
    print(contentString)

最后那些最无聊的事情，才是我最怀念的。
有史以来最强壮的老爷爷和最轻便的房屋~一群最不靠谱的狗和一只引人发笑的怪鸟~外加一位惹人喜爱却戏份不多的老奶奶，组成了一部我大爱的好片！
我心中的结局：就在老爷爷被勒令搬进养老院的第二天，来接他的人发现老爷爷静静地坐在椅子上，手中拿着还没充完的气球。他已经死了，脸上留着幸福的笑容。飞屋冒险只是老爷爷死前的幻境。至此影片结束。
最让我感动的是电影开头，这对夫妻志趣相投、相濡以沫，我想自己能不能找到这样的另一半呢
那老头为啥不早点买机票带老太太去仙境瀑布？
从头哭到尾
前面几分钟就让人落泪，后面却越来越不靠谱
史上第二彪悍钉子户的离奇拆迁故事。Pixar完美地成熟了。
找回丟失已久的夢想吧！在我還未老去的時候···
爱和梦想，勇气和希望
“我总是记得那些无聊的小事情……”“感谢你给了我精彩的一生”，简单并深情着
去掉小孩，坏蛋，狗，鸟的话，这会是部完美的短片，but......;
那段夫妻蒙太奇应该是我迄今最喜欢的pixar段落了;
有一点不得不佩服pixar，拍的第一部3D就能完全摒弃滥斛噱头；而片头前那个“国内首部真人3D电影”的三分钟预告片，足以列入反面教材的经典了


在影片最开头对老夫妻一生的回顾无疑是最大泪点，他们过了很好很幸福的一生，当垂垂老矣，他们仍深爱着对方，就算你已不在，也要带着他们曾经共同的梦想在生命的最终冒险一把，就像小朋友不畏惧一切的好奇心，对生活对梦想，再大的年纪都不是阻碍
气球升起的时候太美好了
基本是儿童剧，最感人的部分一直到屋子初飞起的那一刻，之后就一直在梦游了..=。= 原本以为一切奇想不过是卡尔在再遇女孩那晚所做的一场梦，并且第二天天亮她还会来找他的…对男孩子气的艾莉的印象实在是太深了，这片要是只选取到艾莉死时，卡尔飞起气球，就是满分。皮克斯的技术没得说，因此4星
近乎无可挑剔..
PIXAR总是这么perfect.原来和爱人相守一生是最大的冒险
几十年的夫妻生活浓缩成几分钟那段美好死了。
触动内心最柔软的温情
梦想，多久开始实现都不晚。《冰河世纪》也好，《史瑞克》也好，永远都赶不上皮克斯动画的情怀与意境。


通过前面的学习，我们就完成了网页中短评内容的提取。

明天我们将继续学习，完成短评语句的分词处理，最后生成词云图。

# 百词斩题目

## 怦然心动     

静静最喜欢的电影名字叫“怦然心动”，她想要通过学到的知识，查询这部电影是否进入了豆瓣电影的top25。豆瓣top25电影的网页URL地址是https://movie.douban.com/top250?start=0&filter=
，这该怎么实现呢？

解题思路：
1. 获取User-Agent，设置请求头；
2. 将url和headers参数，添加进requests.get()中，获取网页HTML代码；
3. 创建一个BeautifulSoup对象；
4. 使用find_all()函数找到所有的class="title"节点；
5. 遍历并使用.string属性获取电影名字，判断电影名为怦然心动时，输出"怦然心动是豆瓣top25电影"。

User-Agent：Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36

In [6]:
# TODO 使用import导入requests模块
import requests

# TODO 从bs4中导入BeautifulSoup模块
from bs4 import BeautifulSoup

# TODO 将豆瓣电影评论URL地址，赋值给变量url
url = "https://movie.douban.com/top250?start=0&filter="

# TODO 将User-Agent以字典键对形式赋值给headers
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}

# TODO 将 url 和 headers参数，添加进requests.get()中，将字典headers传递headers参数，给赋值给response
response = requests.get(url, headers = headers)

# TODO 将服务器响应内容转换为字符串形式，赋值给html
html = response.text

# TODO 使用BeautifulSoup()传入变量html和解析器lxml，赋值给soup
soup = BeautifulSoup(html, "lxml")

# TODO 使用find_all()查询soup中class="title"的节点，赋值给content_all
content_all = soup.find_all(class_ = "title")

# TODO for循环遍历content_all
for content in content_all:
    # TODO 获取每个节点中标签内的内容，赋值给contentString
    contentString = content.string

    # TODO 使用if判断contentString等于"怦然心动"时
    if contentString == "怦然心动":
        # TODO 使用print()输出"怦然心动是豆瓣top25电影"
        print("怦然心动是豆瓣top25电影")

怦然心动是豆瓣top25电影


## 电影奖项     

静静在知乎搜索韩国电影《寄生虫》的时候，看到了这部电影收获了许多的奖项。于是，她想利用刚刚学过的爬虫小知识，获取该电影获得的所有的奖项。

已知简介的节点是class="TopicMovieIntro-awardItemTitle"。

URL链接：https://www.zhihu.com/topic/20753544/intro

User-Agent：Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36

注意：本题需要用到.text方法知识点。

In [7]:
# TODO 使用import导入requests模块
import requests

# TODO 从bs4中导入BeautifulSoup模块
from bs4 import BeautifulSoup

# TODO 将"https://www.zhihu.com/topic/20753544/intro"，赋值给变量url
url = "https://www.zhihu.com/topic/20753544/intro"

# TODO 将User-Agent以字典键对形式赋值给headers
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}

# TODO 将 url 和 headers参数，添加进requests.get()中，将字典headers传递headers参数，给赋值给response
response = requests.get(url, headers = headers)

# TODO 将服务器响应内容转换为字符串形式，赋值给html
html = response.text

# TODO 使用BeautifulSoup()传入变量html和解析器lxml，赋值给soup
soup = BeautifulSoup(html, "lxml")

# TODO # 使用find_all()查询soup中class="TopicMovieIntro-awardItemTitle"的节点，赋值给content_all
content_all = soup.find_all(class_ = "TopicMovieIntro-awardItemTitle")

# TODO for循环遍历content_all
for content in content_all:

    # TODO 使用.text方法获取每个节点中标签内容，赋值给contentString
    contentString = content.text
    
    # TODO 输出contentString
    print(contentString)

第13届底特律影评人协会奖
第14届亚洲电影大奖
第18届华盛顿影评人协会奖
第20届美国电影学会奖
第22届英国独立电影奖
第23届好莱坞电影奖
第23届美国在线影评人协会奖
第24届美国艺术指导工会奖
第24届金卫星奖
第25届评论家选择奖
第26届美国演员工会奖
第31届美国制片人工会奖
第32届芝加哥影评人协会奖
第35届独立精神奖
第40届伦敦影评人协会奖
第40届波士顿影评人协会奖
第40届韩国青龙电影奖
第45届法国电影凯撒奖
第45届洛杉矶影评人协会奖
第49届鹿特丹国际电影节
第4届亚特兰大影评人协会奖
第54届美国国家影评人协会奖
第56届韩国电影大钟奖
第56届韩国百想艺术大赏
第67届美国音效剪辑协会奖
第70届美国剪辑工会奖
第72届戛纳电影节
第72届美国导演工会奖
第72届美国编剧工会奖
第73届英国电影和电视艺术学院奖
第77届金球奖
第85届纽约影评人协会奖
第91届美国国家评论协会奖
第92届奥斯卡金像奖


## 每日新闻      

静静要爬取网易的每日新闻标题并输出内容，链接
http://nocturne.bczcdn.com/zip/1625207762993_63705/web.html
每个新闻的标题都嵌套在class="rank"的标签的下一级a标签。

【题目要求】
1. 有的tr标签中内容是空的，所以找到class="rank"标签；
2. 找到class="rank"标签内的a标签；
3. 使用.string获取所有的文字内容并输出。

注意：由于 class 在 Python 里是一个关键字 ，为了避免关键词重复，所以后面需要加一个下划线，即 class_="rank"，这里节点先使用find_all(class_="rank")。

```
# TODO 使用import导入requests模块
import requests

# TODO 使用from...import从bs4中导入BeautifulSoup模块
from bs4 import BeautifulSoup

# TODO 将URL地址赋值给变量url
url = "http://nocturne.bczcdn.com/zip/1625207762993_63705/web.html"

# TODO 将变量url传入requests.get()，赋值给response
response = requests.get(url)

# TODO 使用.text将服务器响应内容转换为字符串形式，赋值给html
html = response.text

# TODO 使用BeautifulSoup()读取html，添加lxml解析器，赋值给soup
soup = BeautifulSoup(html, "lxml")

# TODO 使用find_all()查询soup中class="rank"节点，赋值给content_all
content_all = soup.find_all(class_ = "rank")

# TODO for循环遍历content_all
for content in content_all:

    # TODO 使用find_all()查询name="a"的节点，赋值给title_list
    title_list = content.find_all(name = "a")

    # TODO for循环遍历title_list
    for title in title_list:
        # TODO 使用print()输出标签中的内容
        print(title.string)
```